In [10]:
import spartan as st
from Fraudar import Fraudar
import imp
import Fraudar
imp.reload(Fraudar)

<module 'Fraudar' from 'c:\\Users\\THUEEwl\\Desktop\\Graph-Mining-Algorithms\\Fraudar.py'>

In [23]:
from scipy.sparse import coo_matrix,lil_matrix,csc_matrix,csr_matrix
import numpy as np

In [2]:
tensor = st.loadTensor(path = "inputData/plain_graph_small.zip", col_types = [int, int], sep=' ')
stensor = tensor.toSTensor(hasvalue=False)

In [4]:
stensor._data

Format,coo
Data Type,int64
Shape,"(500, 497)"
nnz,898
Density,0.003613682092555332
Read-only,True
Size,14.0K
Storage ratio,0.0


In [8]:
fr = Fraudar(stensor)

In [19]:
tensor = stensor.to_scipy().asfptype().tolil()

In [20]:
tensor

<500x497 sparse matrix of type '<class 'numpy.float64'>'
	with 898 stored elements in List of Lists format>

In [24]:
def logWeightedAveDegree(M):
    # M: scipy sparse matrix
    (m, n) = M.shape
    colSums = M.sum(axis=0)
    colWeights = 1.0 / np.log(np.squeeze(colSums.A) + 5)
    colDiag = lil_matrix((n, n))
    colDiag.setdiag(colWeights)
    W = M * colDiag
    print("finished computing weight matrix")
    return W


In [25]:
w = logWeightedAveDegree(tensor)

finished computing weight matrix


In [26]:
w

<500x497 sparse matrix of type '<class 'numpy.float64'>'
	with 898 stored elements in Compressed Sparse Row format>

In [11]:
def fastGreedyDecreasing(self, M):
        (m, n) = M.shape
        Ml = M.tolil()
        Mlt = M.transpose().tolil()
        rowSet = set(range(0, m))
        colSet = set(range(0, n))
        curScore = M[list(rowSet), :][:, list(colSet)].sum(axis=None)
        bestAveScore = curScore / (len(rowSet) + len(colSet))
        bestSets = (rowSet, colSet)
        print("finished setting up greedy")
        # *decrease* in total weight when *removing* this row
        # Prepare the min priority tree to begin greedy algorithm.
        rowDeltas = np.squeeze(M.sum(axis=1).A)
        colDeltas = np.squeeze(M.sum(axis=0).A)
        print("finished setting deltas")
        rowTree = MinTree(rowDeltas)
        colTree = MinTree(colDeltas)
        print("finished building min trees")

        numDeleted = 0
        deleted = []
        bestNumDeleted = 0

        while rowSet and colSet:
            if (len(colSet) + len(rowSet)) % 100000 == 0:
                print("current set size = ", len(colSet) + len(rowSet))
            nextRow, rowDelt = rowTree.getMin()
            nextCol, colDelt = colTree.getMin()
            if rowDelt <= colDelt:
                curScore -= rowDelt
                # Update priority for the node with min priority and its neighbors
                for j in Ml.rows[nextRow]:
                    delt = Ml[nextRow, j]
                    colTree.changeVal(j, -delt)
                rowSet -= {nextRow}
                rowTree.changeVal(nextRow, float('inf'))
                deleted.append((0, nextRow))
            else:
                curScore -= colDelt
                # Update priority for the node with min priority and its neighbors
                for i in Mlt.rows[nextCol]:
                    delt = Ml[i, nextCol]
                    rowTree.changeVal(i, -delt)
                colSet -= {nextCol}
                colTree.changeVal(nextCol, float('inf'))
                deleted.append((1, nextCol))

            numDeleted += 1
            curAveScore = curScore / (len(colSet) + len(rowSet))
            if curAveScore > bestAveScore:
                bestAveScore = curAveScore
                bestNumDeleted = numDeleted

        # Reconstruct the best row and column sets
        finalRowSet = set(range(m))
        finalColSet = set(range(n))
        for i in range(bestNumDeleted):
            if deleted[i][0] == 0:
                finalRowSet.remove(deleted[i][1])
            else:
                finalColSet.remove(deleted[i][1])
        return (finalRowSet, finalColSet, bestAveScore)

finished computing weight matrix


TypeError: fastGreedyDecreasing() takes 1 positional argument but 2 were given